In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json


In [ ]:
df = pd.read_csv('http://bit.ly/imdbratings')

df['actors_list'] = df['actors_list'].apply(lambda x: ' '.join(eval(x)))
df['combined_features'] = df['title'] + ' ' + df['genre'] + ' ' + df['actors_list']

tfidf_vectorizer = TfidfVectorizer(stop_words='english')

tfidf_matrix = tfidf_vectorizer.fit_transform(df['combined_features'])

In [ ]:
def get_user_data():
    user_data = {
        "favorite_movies": [],
        "favorite_actors": [],
        "favorite_genres": []
    }

    print("\nYou will now be prompted to provide three of your favourite movies, \n"
          "three of your favourite genres, and three of your favourite actors.\n")

    for i in range(3):
        movie = input("Enter one of your favorite movies: ")
        user_data["favorite_movies"].append(movie)

    for i in range(3):
        genre = input("Enter one of your favorite genres: ")
        user_data["favorite_genres"].append(genre)

    for i in range(3):
        actor = input("Enter one of your favorite actors: ")
        user_data["favorite_actors"].append(actor)

    return user_data

In [ ]:
def get_recommendation(user_data, df):
    user_features = ' '.join(user_data['favorite_movies'] + user_data['favorite_actors'] + user_data['favorite_genres'])
    user_matrix = tfidf_vectorizer.transform([user_features])
    user_cosine_sim = cosine_similarity(user_matrix, tfidf_matrix)
    movie_indices = user_cosine_sim.argsort()[0][::-1]
    recommended_movies = [df.iloc[idx]['title'] for idx in movie_indices if df.iloc[idx]['title'] not in user_data['favorite_movies']]
    return recommended_movies[:10]

In [ ]:
def save_recommendation_history(user_data, recommended_movies):
    history_entry = {
        "user_data": user_data,
        "recommendations": recommended_movies
    }

    try:
        with open("recommendation_history.json", "r") as file:
            history_data = json.load(file)
    except FileNotFoundError:
        history_data = []

    history_data.append(history_entry)

    with open("recommendation_history.json", "w") as file:
        json.dump(history_data, file, indent=2)

In [ ]:
def display_recommendation_history():
    try:
        with open("recommendation_history.json", "r") as file:
            history_data = json.load(file)
            for idx, entry in enumerate(history_data, start=1):
                print(f"\nRecommendation {idx}:")
                print("User Preferences:")
                print(f"Favourite Movies: {', '.join(entry['user_data']['favorite_movies'])}")
                print(f"Favourite Actors: {', '.join(entry['user_data']['favorite_actors'])}")
                print(f"Favourite Genres: {', '.join(entry['user_data']['favorite_genres'])}")
                print("\nMovie Recommendations:")
                for i, movie in enumerate(entry['recommendations'], start=1):
                    print(f"{i}. {movie}")
                print("---------------------------------------------------------------\n")
    except FileNotFoundError:
        print("No recommendation history found.")

In [ ]:
def main():
    user_data = get_user_data()
    recommended_movies = get_recommendation(user_data, df)
    print("\nRecommended Movies:")
    for i, movie in enumerate(recommended_movies, start=1):
        print(f"{i}. {movie}")
    save_recommendation_history(user_data, recommended_movies)


In [ ]:
def menu():
    while True:
        print("\nPersonalized Movie Recommendations Menu:")
        print("1. Get Recommendations")
        print("2. See Recommendation History")
        print("3. Exit")

        choice = input("\n Enter your choice (1-3): \n")

        if choice == '1':
            main()
            while True:
                another_recommendation = input("Do you want another recommendation? (y/n): ")
                if another_recommendation.lower() == 'y':
                    main()
                elif another_recommendation.lower() == 'n':
                    break
                else:
                    print("Invalid input. Please enter 'y' or 'n'.")
        elif choice == '2':
            display_recommendation_history()
        elif choice == '3':
            print("Exiting the program.")
            break
        else:
            print("Invalid choice. Please enter a number between 1 and 3.")



In [ ]:
if __name__ == "__main__":
    menu()


Personalized Movie Recommendations Menu:
1. Get Recommendations
2. See Recommendation History
3. Exit

 Enter your choice (1-3): 
1

You will now be prompted to provide three of your favourite movies, 
three of your favourite genres, and three of your favourite actors.

Enter one of your favorite movies:  One Flew Over the Cuckoo's Nest
Enter one of your favorite movies: Grave of the Fireflies
Enter one of your favorite movies: Braveheart
Enter one of your favorite genres: Crime
Enter one of your favorite genres: Sci-Fi
Enter one of your favorite genres: Mystery
Enter one of your favorite actors: Samuel L. Jackson
Enter one of your favorite actors: Leonardo DiCaprio
Enter one of your favorite actors: Peter Lorre

Recommended Movies:
1. One Flew Over the Cuckoo's Nest
2. M
3. Shutter Island
4. Pulp Fiction
5. The Departed
6. Jackie Brown
7. Gravity
8. The Incredibles
9. Brazil
10. Catch Me If You Can
Do you want another recommendation? (y/n): n

Personalized Movie Recommendations Menu: